<a href="https://colab.research.google.com/github/HemaGarima/Real_state_Price_Prediction/blob/main/Real_State_Price_Prediction_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Regression Project : Predicting Home Prices in Bangalore

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams["figure.figsize"] = (20,10)

## Data Load : Load bangalore home prices into a dataframe

In [ ]:
df1 = pd.read_csv("bengaluru_house_prices.csv")
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.columns

In [ ]:
df1['area_type'].unique()

In [ ]:
df1['area_type'].nunique()

In [ ]:
df1['area_type'].value_counts()

- Drop features that are not required to build our model

In [ ]:
df2 = df1.drop(['area_type' , 'society' , 'balcony' , 'availability'],axis = 'columns')
df2.shape

In [ ]:
df2.head()

## Data Cleaning : Handling NA values

In [ ]:
df2.isnull().sum()

In [ ]:
df2.shape

In [ ]:
df3 = df2.dropna()

In [ ]:
df3.isnull().sum()

In [ ]:
df3.shape

In [ ]:
df3['size'].nunique()

In [ ]:
df3['size'].unique()

In [ ]:
df3['bhk'] = df3['size'].apply(lambda x : int(x.split(' ')[0]))

In [ ]:
df3.head()

In [ ]:
df3['bhk'].unique()

In [ ]:
df3[df3.bhk > 20]

In [ ]:
df3.total_sqft.unique()

In [ ]:
df3['total_sqft'].nunique()

In [ ]:
df3['total_sqft'].unique()

In [ ]:
def is_float(x):
  try:
    float(x)
  except:
    return False
  return True

In [ ]:
df3[~df3['total_sqft'].apply(is_float)]

In [ ]:
df3[~df3['total_sqft'].apply(is_float)].head(10)

- Above shows that total_sqft can be a range (e.g. 2100 - 2850). For such case we can just take average of min and max value in the range. There are other cases such as 34.46sq.Meter which one can convert to square ft using unit conversion.

In [ ]:
def convert_sqft_to_num(x):
  tokens = x.split('-')
  if len(tokens) == 2:
    return (float(tokens[0]) + float(tokens[1]))/2
  try:
    return float(x)
  except:
    return None

In [ ]:
convert_sqft_to_num('2166')

In [ ]:
convert_sqft_to_num('2100 - 2850')

In [ ]:
convert_sqft_to_num('34.46Sq. Meter')

In [ ]:
df4 = df3.copy()
df4.total_sqft = df4.total_sqft.apply(convert_sqft_to_num)
df4 = df4[df4.total_sqft.notnull()]
df4.head()

In [ ]:
df4.loc[30]

In [74]:
2100+2850

4950

In [75]:
4950/2

2475.0

In [76]:
df4.shape

(13200, 6)

## Feature Engineering

- Add new feature called price per square feet

In [77]:
df5 = df4.copy()
df5['price_per_sqft'] = df5['price']*100000 / df5['total_sqft']
df5.head()

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000


In [78]:
df5_stats = df5['price_per_sqft'].describe()
df5_stats

,price_per_sqft
count,1.320000e+04
mean,7.920759e+03
std,1.067272e+05
min,2.678298e+02
25%,4.267701e+03
50%,5.438331e+03
75%,7.317073e+03
max,1.200000e+07


In [79]:
df5.to_csv("bhp.csv",index = False)

- Examine locations which is a categorical variable. We need to apply dimensionality reduction technique here to reduce number of locations

In [80]:
df5.location = df5.location.apply(lambda x : x.strip())

In [82]:
location_stats = df5['location'].value_counts(ascending = False)
location_stats

,count
location,
Whitefield,533
Sarjapur Road,392
Electronic City,304
Kanakpura Road,264
Thanisandra,235
...,...
Rajanna Layout,1
Subramanyanagar,1
Lakshmipura Vidyaanyapura,1


In [83]:
location_stats.values.sum()

13200

In [84]:
df5.shape

(13200, 7)

In [85]:
len(location_stats[location_stats > 10])

240

In [86]:
len(location_stats)

1287

In [87]:
len(location_stats[location_stats <= 10])

1047

## Dimensionality Reduction

- Any location having less than 10 data points should be tagged as "other" location. This way number of categories can be reduced by huge amount. Later on when we do one hot encoding, it will help us with having fewer dummy columns


In [89]:
location_stats_less_than_10 = location_stats[location_stats <= 10]
location_stats_less_than_10

,count
location,
BTM 1st Stage,10
Gunjur Palya,10
Nagappa Reddy Layout,10
Sector 1 HSR Layout,10
Thyagaraja Nagar,10
...,...
Rajanna Layout,1
Subramanyanagar,1
Lakshmipura Vidyaanyapura,1


In [90]:
len(df5.location.unique())

1287

In [91]:
df5.location.nunique()

1287

In [92]:
df5.location = df5.location.apply(lambda x : 'other' if x in location_stats_less_than_10 else x)

In [93]:
len(df5.location.unique())

241

In [94]:
df5.head(10)

,location,size,total_sqft,bath,price,bhk,price_per_sqft
0,Electronic City Phase II,2 BHK,1056.0,2.0,39.07,2,3699.810606
1,Chikka Tirupathi,4 Bedroom,2600.0,5.0,120.00,4,4615.384615
2,Uttarahalli,3 BHK,1440.0,2.0,62.00,3,4305.555556
3,Lingadheeranahalli,3 BHK,1521.0,3.0,95.00,3,6245.890861
4,Kothanur,2 BHK,1200.0,2.0,51.00,2,4250.000000
5,Whitefield,2 BHK,1170.0,2.0,38.00,2,3247.863248
6,Old Airport Road,4 BHK,2732.0,4.0,204.00,4,7467.057101
7,Rajaji Nagar,4 BHK,3300.0,4.0,600.00,4,18181.818182
8,Marathahalli,3 BHK,1310.0,3.0,63.25,3,4828.244275
9,other,6 Bedroom,1020.0,6.0,370.00,6,36274.509804
